# libraries

In [1]:
!apt update
!apt install ffmpeg libsm6 libxext6 -y
!pip install opencv-python
!pip install --upgrade pip

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://deb.debian.org/debian buster/main amd64 Packages [7,909 kB]
Get:5 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [473 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [8,788 B]
Fetched 8,604 kB in 2s (3,779 kB/s)



13 packages can be upgraded. Run 'apt list --upgradable' to see them.



libsm6 is already the newest version (2:1.2.3-1).
libsm6 set to manually installed.
libxext6 is already the newest version (2:1.3.3-1+b2).
libxext6 set to manually installed.
The following additional packages will be installed:
  i965-va-driver intel-media-va-driver libaacs0 libaom0 libasound2
  libasound2-data libass9 libasyncns0 libavc1394-0 libavcodec58 libavdevice58
  libavfilter7 libavformat58 libavresample4 libavutil56 libbdplu

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/datasets/deepnote/MSC-NLP-PROJECT/train')

label_types = os.listdir('/datasets/deepnote/MSC-NLP-PROJECT/train')
print (label_types)

['calm', 'curious', 'cry', 'angry', 'bored', 'afraid', 'care', 'confused', 'admire', 'concern', 'disgust', 'bad', 'comfortable', 'annoy', 'depressed', 'beautiful']


# Video classifier using cnn and rnn

In [3]:
videos = []

for item in dataset_path:
 # Get all the file names
 all_videos = os.listdir('/datasets/deepnote/MSC-NLP-PROJECT/train' + '/' +item)

 # Add them to the list
 for video in all_videos:
    videos.append((item, str('/datasets/deepnote/MSC-NLP-PROJECT/train' + '/' +item) + '/' + video))
    
# Build a dataframe        
train_df = pd.DataFrame(data=videos, columns=['emotion', 'video_name'])
print(train_df.head())
print(train_df.tail())

   emotion                                         video_name
0  curious  /datasets/deepnote/MSC-NLP-PROJECT/train/curio...
1  curious  /datasets/deepnote/MSC-NLP-PROJECT/train/curio...
2  curious  /datasets/deepnote/MSC-NLP-PROJECT/train/curio...
3  curious  /datasets/deepnote/MSC-NLP-PROJECT/train/curio...
4  curious  /datasets/deepnote/MSC-NLP-PROJECT/train/curio...
    emotion                                         video_name
100    care  /datasets/deepnote/MSC-NLP-PROJECT/train/care/...
101    care  /datasets/deepnote/MSC-NLP-PROJECT/train/care/...
102    care  /datasets/deepnote/MSC-NLP-PROJECT/train/care/...
103    care  /datasets/deepnote/MSC-NLP-PROJECT/train/care/...
104    care  /datasets/deepnote/MSC-NLP-PROJECT/train/care/...


In [4]:
df = train_df.loc[:,['video_name','emotion']]
df
df.to_csv('train.csv')

# preparing the test data

In [5]:
folder_path_test = '/datasets/deepnote/MSC-NLP-PROJECT/test'

if os.path.exists(folder_path_test):
   label_types = os.listdir(folder_path_test)
   print (label_types) 
else:
    print("Folder does not exist.")

#print (label_types)  

['cry', 'curious', 'confused', 'concern', 'afraid', 'annoy', 'depressed', 'angry', 'bored', 'bad', 'disgust', 'admire', 'calm', 'care', 'beautiful', 'comfortable']


In [6]:
dataset_path = os.listdir('/datasets/deepnote/MSC-NLP-PROJECT/test')
print(dataset_path)

room_types = os.listdir('/datasets/deepnote/MSC-NLP-PROJECT/test')
print("Some words found: ", len(dataset_path))

videos = []

for item in dataset_path:
 # Get all the file names
 all_videos = os.listdir('/datasets/deepnote/MSC-NLP-PROJECT/test' + '/' +item)

 # Add them to the list
 for video in all_videos:
    videos.append((item, str('/datasets/deepnote/MSC-NLP-PROJECT/test' + '/' +item) + '/' + video))
    
# Build a dataframe        
test_df = pd.DataFrame(data=videos, columns=['emotion', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','emotion']]
df
df.to_csv('test.csv')

['comfortable', 'bad', 'curious', 'afraid', 'calm', 'bored', 'care', 'concern', 'cry', 'disgust', 'confused', 'depressed', 'annoy', 'beautiful', 'angry', 'admire']
Some words found:  16
       emotion                                         video_name
0  comfortable  /datasets/deepnote/MSC-NLP-PROJECT/test/comfor...
1  comfortable  /datasets/deepnote/MSC-NLP-PROJECT/test/comfor...
2          bad  /datasets/deepnote/MSC-NLP-PROJECT/test/bad/ba...
3          bad  /datasets/deepnote/MSC-NLP-PROJECT/test/bad/ba...
4          bad  /datasets/deepnote/MSC-NLP-PROJECT/test/bad/ba...
      emotion                                         video_name
21  beautiful  /datasets/deepnote/MSC-NLP-PROJECT/test/beauti...
22  beautiful  /datasets/deepnote/MSC-NLP-PROJECT/test/beauti...
23      angry  /datasets/deepnote/MSC-NLP-PROJECT/test/angry/...
24      angry  /datasets/deepnote/MSC-NLP-PROJECT/test/angry/...
25     admire  /datasets/deepnote/MSC-NLP-PROJECT/test/admire...


# tensoflo lib

In [7]:
import cv2

In [8]:
from tensorflow_docs.vis import embed
from tensorflow import keras
#from imutils import paths


import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio

import os



2023-03-31 12:00:44.278787: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 12:00:44.487335: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /root/venv/lib/python3.9/site-packages/cv2/../../lib64:
2023-03-31 12:00:44.487399: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-31 12:00:44.537165: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regis

In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)



2023-03-31 12:00:51.773275: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /root/venv/lib/python3.9/site-packages/cv2/../../lib64:
2023-03-31 12:00:51.773325: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-31 12:00:51.773351: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-69bec130-7c4b-403a-9150-8c0547a03d09): /proc/driver/nvidia/version does not exist


In [10]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(100)



Total videos for training: 105
Total videos for testing: 26


,Unnamed: 0,video_name,emotion
81,81,/datasets/deepnote/MSC-NLP-PROJECT/train/bad/b...,bad
65,65,/datasets/deepnote/MSC-NLP-PROJECT/train/calm/...,calm
60,60,/datasets/deepnote/MSC-NLP-PROJECT/train/calm/...,calm
77,77,/datasets/deepnote/MSC-NLP-PROJECT/train/bad/b...,bad
103,103,/datasets/deepnote/MSC-NLP-PROJECT/train/care/...,care
...,...,...,...
16,16,/datasets/deepnote/MSC-NLP-PROJECT/train/comfo...,comfortable
29,29,/datasets/deepnote/MSC-NLP-PROJECT/train/depre...,depressed
0,0,/datasets/deepnote/MSC-NLP-PROJECT/train/curio...,curious
12,12,/datasets/deepnote/MSC-NLP-PROJECT/train/angry...,angry


In [11]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(200, 200)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

# feature extraction

In [12]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(200, 200, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((200, 200, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

2023-03-31 12:00:51.927978: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
87910968/87910968 [==============================] - 1s 0us/step


# label encoding

In [13]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["emotion"]))
print(label_processor.get_vocabulary())

labels = train_df["emotion"].values
labels = label_processor(labels[..., None]).numpy()
labels



['admire', 'afraid', 'angry', 'annoy', 'bad', 'beautiful', 'bored', 'calm', 'care', 'comfortable', 'concern', 'confused', 'cry', 'curious', 'depressed', 'disgust']


array([[13],
       [13],
       [13],
       [13],
       [13],
       [13],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 9],
       [ 9],
       [ 9],
       [ 9],
       [ 9],
       [ 9],
       [ 9],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [ 5],
       [14],
       [14],
       [14],
       [14],
       [14],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [10],
       [10],
       [10],
       [10],
       [10],
       [11],
       [11],
       [11],
       [11],
       [11],
       [11],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 7],
       [ 7],
       [ 7],
       [ 7],
       [ 7],
       [ 7],
       [ 7],
       [ 7],
       [ 7],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 4],
       [ 4],
       [ 4],
       [ 4],
       [ 4],

# hyperparameters

In [14]:
IMG_SIZE = 200
BATCH_SIZE = 64
EPOCHS = 50

MAX_SEQ_LENGTH = 15
NUM_FEATURES = 2048

In [15]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(100)


Total videos for training: 105
Total videos for testing: 26


,Unnamed: 0,video_name,emotion
52,52,/datasets/deepnote/MSC-NLP-PROJECT/train/annoy...,annoy
59,59,/datasets/deepnote/MSC-NLP-PROJECT/train/calm/...,calm
103,103,/datasets/deepnote/MSC-NLP-PROJECT/train/care/...,care
62,62,/datasets/deepnote/MSC-NLP-PROJECT/train/calm/...,calm
73,73,/datasets/deepnote/MSC-NLP-PROJECT/train/bad/b...,bad
...,...,...,...
86,86,/datasets/deepnote/MSC-NLP-PROJECT/train/disgu...,disgust
53,53,/datasets/deepnote/MSC-NLP-PROJECT/train/afrai...,afraid
58,58,/datasets/deepnote/MSC-NLP-PROJECT/train/calm/...,calm
7,7,/datasets/deepnote/MSC-NLP-PROJECT/train/angry...,angry


In [16]:

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(200, 200)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [17]:
IMG_SIZE = 200
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()


In [18]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["emotion"])
)
print(label_processor.get_vocabulary())


['admire', 'afraid', 'angry', 'annoy', 'bad', 'beautiful', 'bored', 'calm', 'care', 'comfortable', 'concern', 'confused', 'cry', 'curious', 'depressed', 'disgust']


In [19]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["emotion"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")


print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

1/1 [==============================] - 0s 271ms/step
[h264 @ 0x5558f763ce00] Invalid NAL unit size (745 > 472).
[h264 @ 0x5558f763ce00] Error splitting the input into NAL units.
1/1 [==============================] - 0s 274ms/step
[h264 @ 0x5558fe8fefc0] Invalid NAL unit size (745 > 472).
[h264 @ 0x5558fe8fefc0] Error splitting the input into NAL units.
1/1 [==============================] - 0s 347ms/step
Frame features in train set: (105, 15, 2048)
Frame masks in train set: (105, 15)
train_labels in train set: (105, 1)
test_labels in train set: (26, 1)


# sequence model

In [20]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 35
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/35
2/3 [===================>..........] - ETA: 0s - loss: 2.9116 - accuracy: 0.1250 
Epoch 1: val_loss improved from inf to 3.24222, saving model to ./tmp/video_classifier
3/3 [==============================] - 18s 2s/step - loss: 2.8571 - accuracy: 0.1233 - val_loss: 3.2422 - val_accuracy: 0.0000e+00
Epoch 2/35
3/3 [==============================] - ETA: 0s - loss: 2.8188 - accuracy: 0.0548
Epoch 2: val_loss improved from 3.24222 to 3.10094, saving model to ./tmp/video_classifier
3/3 [==============================] - 0s 154ms/step - loss: 2.8188 - accuracy: 0.0548 - val_loss: 3.1009 - val_accuracy: 0.0000e+00
Epoch 3/35
2/3 [===================>..........] - ETA: 0s - loss: 2.6863 - accuracy: 0.1406
Epoch 3: val_loss improved from 3.10094 to 3.04575, saving model to ./tmp/video_classifier
3/3 [==============================] - 0s 142ms/step - loss: 2.6801 - accuracy: 0.1233 - val_loss: 3.0458 - val_accuracy: 0.0000e+00
Epoch 4/35
3/3 [==============================] - ETA: 0s

# inference

In [21]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)



Test video path: /datasets/deepnote/MSC-NLP-PROJECT/test/angry/angry.mp4
1/1 [==============================] - 5s 5s/step
  admire:  8.64%
  confused:  8.48%
  calm:  7.82%
  curious:  7.20%
  bored:  7.16%
  angry:  6.78%
  beautiful:  6.72%
  annoy:  6.33%
  cry:  6.14%
  depressed:  5.99%
  concern:  5.66%
  comfortable:  5.54%
  disgust:  5.21%
  bad:  4.31%
  afraid:  4.21%
  care:  3.81%


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=69bec130-7c4b-403a-9150-8c0547a03d09' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>